In [ ]:
import pickle

with open('./res/rag_data.pkl', 'rb') as f:
    rag_data = pickle.load(f)

## 📂 RAG 데이터 불러오기

이전에 저장한 Pickle 파일을 불러와서 메모리에 로드합니다.

- **`pickle.load()`**: 저장된 파이썬 객체를 다시 불러오기
- **`'rb'` 모드**: Read Binary (바이너리 읽기 모드)
- **`rag_data`**: 질문, 컨텍스트, 정답 등이 담긴 딕셔너리
- **목적**: RAG 시스템의 성능을 평가하기 위한 데이터 준비

In [ ]:
rag_data['questions'][0]

## 🔍 첫 번째 질문 확인

데이터의 첫 번째 질문을 출력하여 데이터 구조를 확인합니다.

- **`rag_data['questions'][0]`**: 질문 리스트의 첫 번째 요소
- **예시**: "글로벌 저금리 현상이 부각된 원인은 무엇인가요?"
- **목적**: 어떤 형태의 질문이 들어있는지 확인

In [ ]:
rag_data['contexts'][0]

## 📄 첫 번째 질문의 컨텍스트들 확인

첫 번째 질문에 대한 3개의 후보 컨텍스트(문단)를 출력합니다.

- **`rag_data['contexts'][0]`**: 첫 번째 질문의 컨텍스트 리스트
- **3개의 문단**: 이 중 1개가 정답을 포함하고 있음
- **RAG의 핵심**: 이 3개 중에서 질문과 가장 관련 있는 문단을 찾는 것이 목표

In [ ]:
from utils import get_embedding, cosine_similarity


embed_q = get_embedding(rag_data['questions'][0])
embed_c0 = get_embedding(rag_data['contexts'][0][0])
embed_c1 = get_embedding(rag_data['contexts'][0][1])
embed_c2 = get_embedding(rag_data['contexts'][0][2])

## 🧮 임베딩 생성

질문과 3개의 컨텍스트를 각각 벡터(숫자 배열)로 변환합니다.

- **`get_embedding()`**: 텍스트를 고차원 벡터로 변환하는 함수
- **임베딩이란?**: 텍스트를 컴퓨터가 이해할 수 있는 숫자 배열로 표현
- **목적**: 의미가 비슷한 텍스트는 비슷한 벡터 값을 갖게 됨
- **변환 결과**: 
  - `embed_q`: 질문의 벡터
  - `embed_c0`, `embed_c1`, `embed_c2`: 각 컨텍스트의 벡터

In [ ]:
print(cosine_similarity(embed_q, embed_c0))
print(cosine_similarity(embed_q, embed_c1))
print(cosine_similarity(embed_q, embed_c2))

## 📊 코사인 유사도 계산

질문과 각 컨텍스트 간의 유사도를 계산합니다.

- **`cosine_similarity()`**: 두 벡터 간의 유사도를 0~1 사이 값으로 계산
- **코사인 유사도란?**: 두 벡터의 방향이 얼마나 비슷한지 측정 (1에 가까울수록 유사)
- **출력 결과**: 
  - 첫 번째 컨텍스트와의 유사도
  - 두 번째 컨텍스트와의 유사도
  - 세 번째 컨텍스트와의 유사도
- **해석**: 가장 높은 값을 가진 컨텍스트가 질문과 가장 관련이 깊음

In [ ]:
rag_data['contexts_answer_idx'][0]

## ✅ 정답 컨텍스트 인덱스 확인

첫 번째 질문의 실제 정답이 어느 컨텍스트에 있는지 확인합니다.

- **`contexts_answer_idx[0]`**: 정답이 들어있는 컨텍스트의 위치 (0, 1, 또는 2)
- **목적**: 예측 결과와 비교하기 위한 정답 확인
- **예시**: 만약 값이 2라면, 세 번째 컨텍스트(`contexts[0][2]`)가 정답

In [ ]:
embed_q = get_embedding(rag_data['questions'][0], model='text-embedding-3-large')
embed_c0 = get_embedding(rag_data['contexts'][0][0], model='text-embedding-3-large')
embed_c1 = get_embedding(rag_data['contexts'][0][1], model='text-embedding-3-large')
embed_c2 = get_embedding(rag_data['contexts'][0][2], model='text-embedding-3-large')

print(cosine_similarity(embed_q, embed_c0))
print(cosine_similarity(embed_q, embed_c1))
print(cosine_similarity(embed_q, embed_c2))

## 🚀 더 큰 임베딩 모델 테스트

더 성능이 좋은 임베딩 모델로 같은 작업을 수행합니다.

- **`text-embedding-3-large`**: OpenAI의 더 크고 정확한 임베딩 모델
- **기본 모델 vs Large 모델**: 
  - 기본: 빠르지만 정확도가 낮음
  - Large: 느리지만 더 정확한 의미 파악
- **비교 목적**: 모델 크기가 성능에 어떤 영향을 주는지 확인
- **결과 예상**: Large 모델이 정답 컨텍스트에 더 높은 유사도를 줄 가능성이 높음

In [ ]:
from tqdm import tqdm

num_questions = 10
num_contexts = 3

top_context_indices = []

for i in tqdm(range(num_questions)):
    embed_q = get_embedding(rag_data['questions'][i])

    similarities = []
    for j in range(num_contexts):
        embed_c = get_embedding(rag_data['contexts'][i][j])
        similarity = cosine_similarity(embed_q, embed_c)
        similarities.append(similarity)

    top_context_index = similarities.index(max(similarities))
    top_context_indices.append(top_context_index)

print(f"Top Context Indices: {top_context_indices}")

## 🔄 10개 질문에 대한 자동 평가

처음 10개의 질문에 대해 자동으로 가장 유사한 컨텍스트를 찾습니다.

### 동작 과정
1. **반복문 실행**: 10개 질문을 하나씩 처리
2. **각 질문마다**:
   - 질문을 임베딩으로 변환
   - 3개의 컨텍스트를 각각 임베딩으로 변환
   - 유사도 계산
   - 가장 높은 유사도를 가진 컨텍스트의 인덱스 저장
3. **진행 상황 표시**: `tqdm`으로 진행률 바 출력

### 결과
- **`top_context_indices`**: 각 질문마다 예측한 정답 컨텍스트의 위치 리스트
- **예시**: `[2, 1, 0, 2, 1, ...]` → 첫 번째 질문은 3번째 컨텍스트가 정답이라 예측

In [ ]:
rag_data['contexts_answer_idx'][:num_questions]

## 📋 실제 정답 인덱스 확인

10개 질문의 실제 정답 컨텍스트 위치를 출력합니다.

- **`contexts_answer_idx[:10]`**: 처음 10개 질문의 정답 위치 리스트
- **비교 대상**: 위에서 예측한 `top_context_indices`와 비교할 기준값
- **목적**: 예측이 얼마나 정확한지 평가하기 위한 정답 데이터

In [ ]:
def calculate_accuracy(predicted, actual):
    correct = sum(p == a for p, a in zip(predicted, actual))
    total = len(predicted)
    
    accuracy = correct / total
    return accuracy

accuracy = calculate_accuracy(top_context_indices, rag_data['contexts_answer_idx'][:10])
print(f"Accuracy: {accuracy:.2%}")

## 🎯 정확도 계산

예측 결과와 실제 정답을 비교하여 정확도를 계산합니다.

### 함수 설명
- **`calculate_accuracy()`**: 예측값과 실제값을 비교하여 정확도 계산
- **동작 원리**:
  1. 예측과 정답이 일치하는 개수 세기
  2. 전체 개수로 나누기
  3. 백분율로 변환

### 결과 해석
- **Accuracy: 70%**: 10개 중 7개를 정확하게 예측했다는 의미
- **높을수록 좋음**: 100%에 가까울수록 RAG 시스템이 정확함
- **실전 의미**: 사용자 질문에 올바른 정보를 제공할 확률

In [ ]:
contexts_predictions = []
for i in range(len(top_context_indices)):
    index = top_context_indices[i]
    contexts_predictions.append([rag_data['contexts'][i][index]])
contexts_predictions

## 📦 예측된 컨텍스트 추출

예측한 인덱스를 사용하여 실제 컨텍스트 텍스트를 추출합니다.

### 동작 과정
- **반복문**: 10개 질문의 예측 결과를 순회
- **인덱스로 접근**: 각 질문에서 예측한 위치의 컨텍스트 가져오기
- **리스트로 감싸기**: `[context]` 형태로 변환 (RAGAS 평가 형식 맞추기)

### 결과
- **`contexts_predictions`**: 각 질문마다 예측한 컨텍스트 텍스트가 담긴 리스트
- **형식**: `[[context1], [context2], ...]`
- **목적**: 다음 단계에서 RAGAS로 평가하기 위한 데이터 준비

In [ ]:
from datasets import Dataset 
import os
from ragas import evaluate
from ragas.metrics import faithfulness, answer_correctness, context_relevancy, context_recall, context_precision


data_samples = {
    'question': rag_data['questions'][:num_questions],
    'contexts' : contexts_predictions,
    'ground_truth': rag_data['answers'][:num_questions]
}

dataset = Dataset.from_dict(data_samples)

score = evaluate(dataset, metrics=[context_recall])
score

## 📈 RAGAS 평가 수행

RAGAS 프레임워크를 사용하여 RAG 시스템의 성능을 전문적으로 평가합니다.

### RAGAS란?
- **RAG Assessment**: RAG 시스템을 평가하는 전문 라이브러리
- **다양한 메트릭 제공**: faithfulness, relevancy, recall, precision 등

### 평가 데이터 준비
- **`question`**: 사용자 질문
- **`contexts`**: 예측한 컨텍스트 (우리가 찾은 문단)
- **`ground_truth`**: 실제 정답

### 평가 메트릭
- **`context_recall`**: 정답을 포함하는 컨텍스트를 얼마나 잘 찾았는가
  - 1.0에 가까울수록 좋음
  - 예: 0.8 → 정답 정보의 80%를 찾아냄

### 결과
평가 점수와 함께 RAG 시스템의 검색 성능이 수치로 표시됩니다.